In [1]:
import json
import numpy as np
import os
import pandas as pd
import pysmiles
import shutil as sh
import tensorflow as tf
from pathlib import Path
from tqdm.auto import tqdm

In [2]:

data_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/dsgdb9nsd")
limit = 1
output_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/molecules")
output_fn = "mol_{id}.json"

In [3]:
def empty_dirs(dirs=None):
    if dirs is None:
        return
    elif isinstance(dirs, Path):
        dirs = [Path(dirs)]
    for _dir in dirs:
        assert isinstance(_dir, Path)
        for file in _dir.glob("*"):
            file.unlink()

def process_files(data_dir=Path("/home/rmonge/UPC/TFM/gnn-qm9/data/dsgdb9nsd"), limit=None,
                  output_fn="mol_{id}.json", output_dir=Path("/home/rmonge/UPC/TFM/gnn-qm9/data/molecules"),
                  verbose=False):
    files = list(data_dir.glob("*.xyz"))
    for file in files[:limit]:
        with file.open("r") as fp:
            lines = [l.replace("\n", "").split("\t") for l in fp.readlines()]
        na = int(lines[0][0])
        molecule = {
            "na": na,
            "molecule_props": dict(zip(
                ["id", "rotational_a", "rotational_b", "rotational_c", "dipole_moment", "polarizability",
                 "homo_energy", "lumo_energy", "spatial_extent", "internal_energy_0k", "internal_energy_298k",
                 "free_energy", "heat_capacity"],
                [float(e.replace("gdb", "").strip()) for e in lines[1][:-1]]
            )),
            "coordinates": [
                [c[0], float(c[1].replace("*^", "e")), float(c[2].replace("*^", "e")),
                 float(c[3].replace("*^", "e")), float(c[4].replace("*^", "e"))]
                for c in lines[2:(na+2)]
            ],
            "frequencies": lines[na+2],
            "smiles": lines[na+3][0],
            "inchi": lines[na+4]
        }
        mol_graph = pysmiles.read_smiles(molecule["smiles"], explicit_hydrogen=True)
        edges = list(mol_graph.edges(data=True))
        atoms = list(mol_graph.nodes(data=True))
        graph = {}
        coordinates = molecule["coordinates"]
        # Edge features
        graph["edge_sources"] = []
        graph["edge_targets"] = []
        graph["edge_distance"] = []
        graph["edge_order_1"] = []
        graph["edge_order_1_5"] = []
        graph["edge_order_2"] = []
        graph["edge_order_3"] = []
        graph["atom"] = [str(i) for i in range(len(atoms))]
        graph["adjacency_lists"] = {str(i): [] for i in range(len(atoms))}
        for edge in edges:  # Two edges representing both message passing directions
            start = np.array(coordinates[edge[0]][1:])
            end = np.array(coordinates[edge[1]][1:])
            distance = np.sqrt(np.sum(np.square(end - start)))
            graph["edge_sources"].extend([edge[0], edge[1]])
            graph["edge_targets"].extend([edge[1], edge[0]])
            graph["edge_distance"].extend([distance, distance])
            edge_order = edge[2]["order"]
            graph["edge_order_1"].extend([int(edge_order == 1), int(edge_order == 1)])
            graph["edge_order_1_5"].extend([int(edge_order == 1.5), int(edge_order == 1.5)])
            graph["edge_order_2"].extend([int(edge_order == 2), int(edge_order == 2)])
            graph["edge_order_3"].extend([int(edge_order == 3), int(edge_order == 3)])
            graph["adjacency_lists"][str(edge[0])].append([
                str(edge[1]),
                [distance, int(edge_order == 1), int(edge_order == 1.5), int(edge_order == 2), int(edge_order == 3)]
            ])
            graph["adjacency_lists"][str(edge[1])].append([
                str(edge[0]),
                [distance, int(edge_order == 1), int(edge_order == 1.5), int(edge_order == 2), int(edge_order == 3)]
            ])
        # Node features
        graph["node_x"] = [float(coord[1]) for coord in molecule["coordinates"]]
        graph["node_y"] = [float(coord[2]) for coord in molecule["coordinates"]]
        graph["node_z"] = [float(coord[3]) for coord in molecule["coordinates"]]
        graph["node_e"] = [float(coord[4]) for coord in molecule["coordinates"]]
        graph["node_element_h"] = [1 if atom[1]["element"] == "H" else 0 for atom in atoms]
        graph["node_element_c"] = [1 if atom[1]["element"] == "C" else 0 for atom in atoms]
        graph["node_element_n"] = [1 if atom[1]["element"] == "N" else 0 for atom in atoms]
        graph["node_element_o"] = [1 if atom[1]["element"] == "O" else 0 for atom in atoms]
        graph["node_element_f"] = [1 if atom[1]["element"] == "F" else 0 for atom in atoms]
        graph["node_charge"] = [atom[1]["charge"] for atom in atoms]
        graph["node_aromatic"] = [1 if atom[1]["aromatic"] else 0 for atom in atoms]
        # Targets
        for key in [k for k in molecule["molecule_props"].keys() if k != "id"]:
            graph[key] = float(molecule["molecule_props"][key])
        filename = output_fn.format(id=str(molecule["molecule_props"]["id"]).replace(".", "_"))
        filepath = output_dir / filename
        with filepath.open("w") as fp:
            json.dump(graph, fp)
        if verbose:
            print(f"Processed file {file.name} and saved to {filename}.")

In [4]:
empty_dirs(output_dir)
process_files(data_dir=data_dir, limit=limit, output_fn=output_fn, output_dir=output_dir)

### Tensorflow Datset

In [5]:
mol_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/molecules")
filename = os.listdir(mol_dir)[0]
target = "dipole_moment"

graph = json.load((mol_dir / filename).open())
y = graph[target]
num_edges = len(graph["edge_sources"])
num_node_features = len([key for key in graph if "node_" in key])
num_edge_features = len([key for key in graph if "edge_" in key and not key in ["edge_sources", "edge_targets"]])

print(f"Target: {target}={y}")
for key, value in graph.items():
    if isinstance(value, (list, dict)):
        print(f"- {key:15} length: {len(value)}")
    else:
        print(f"- {key:15} value: {value}")
    
edge_sources = tf.constant(graph.pop("edge_sources"))
edge_targets = tf.constant(graph.pop("edge_targets"))
node_features = tf.stack(
    [tf.constant(values, dtype=float) for key, values in graph.items() if "node_" in key], axis=1
)
edge_features = tf.stack(
    [tf.constant(values, dtype=float) for key, values in graph.items() if "edge_" in key], axis=1
)
data = {
    "node_features": node_features,
    "edge_features": edge_features,
    "edge_sources": edge_sources,
    "edge_targets": edge_targets
}

Target: dipole_moment=1.6252
- edge_sources    length: 42
- edge_targets    length: 42
- edge_distance   length: 42
- edge_order_1    length: 42
- edge_order_1_5  length: 42
- edge_order_2    length: 42
- edge_order_3    length: 42
- atom            length: 20
- adjacency_lists length: 20
- node_x          length: 20
- node_y          length: 20
- node_z          length: 20
- node_e          length: 20
- node_element_h  length: 20
- node_element_c  length: 20
- node_element_n  length: 20
- node_element_o  length: 20
- node_element_f  length: 20
- node_charge     length: 20
- node_aromatic   length: 20
- rotational_a    value: 2.27742
- rotational_b    value: 1.6532
- rotational_c    value: 1.18583
- dipole_moment   value: 1.6252
- polarizability  value: 87.27
- homo_energy     value: -0.2254
- lumo_energy     value: 0.0204
- spatial_extent  value: 0.2458
- internal_energy_0k value: 1128.2239
- internal_energy_298k value: 0.169036
- free_energy     value: -365.942504
- heat_capacity   v

In [6]:
def get_data(molecules_fn, target="dipole_moment"):
    """Data generator for molecule JSON files into GNNInput namedtuples."""
    for fn in molecules_fn:
        with fn.open("r") as fp:
            graph = json.load(fp)
        edge_sources = tf.constant(graph.pop("edge_sources"))
        edge_targets = tf.constant(graph.pop("edge_targets"))
        node_features = tf.stack(
            [tf.constant(values, dtype=float) for key, values in graph.items() if "node_" in key], axis=1
        )
        edge_features = tf.stack(
            [tf.constant(values, dtype=float) for key, values in graph.items() if "edge_" in key], axis=1
        )
        data = {
            "node_features": node_features,
            "edge_features": edge_features,
            "edge_sources": edge_sources,
            "edge_targets": edge_targets
        }
        target = tf.constant([graph[target]])
        yield data, target

In [7]:
data_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/molecules")
molecules_fn = list(data_dir.glob("*.json"))
target = "dipole_moment"
dataset = tf.data.Dataset.from_generator(
    lambda: get_data(molecules_fn, target),
    output_types=(
        {
            "node_features": tf.float32,
            "edge_features": tf.float32,
            "edge_sources": tf.int32,
            "edge_targets": tf.int32
        }, tf.float32
    ),
    output_shapes=(
        {
            "node_features": tf.TensorShape([None, num_node_features]),
            "edge_features": tf.TensorShape([None, num_edge_features]),
            "edge_sources": tf.TensorShape([None]),
            "edge_targets": tf.TensorShape([None])
        }, tf.TensorShape([1])
    )
)
dataset

<FlatMapDataset shapes: ({node_features: (None, 11), edge_features: (None, 5), edge_sources: (None,), edge_targets: (None,)}, (1,)), types: ({node_features: tf.float32, edge_features: tf.float32, edge_sources: tf.int32, edge_targets: tf.int32}, tf.float32)>

In [8]:
graph, y = list(dataset.take(1))[0]
y, graph

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.6252], dtype=float32)>,
 {'node_features': <tf.Tensor: shape=(20, 11), dtype=float32, numpy=
  array([[-0.0521441 ,  1.567143  ,  0.01869116, -0.472714  ,  0.        ,
           1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.01139035,  0.0524598 ,  0.08324553,  0.293066  ,  0.        ,
           1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [-1.254283  , -0.6110634 , -0.42909187, -0.452485  ,  0.        ,
           1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 1.2056264 , -0.71944237, -0.3237049 , -0.209076  ,  0.        ,
           0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 2.4314919 ,  0.04750634,  0.09159327, -0.156554  ,  0.        ,
           1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
      

# Training and test datasets

In [9]:
training_molecules = 10000
test_molecules = 10000
limit = training_molecules + test_molecules
data_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/dsgdb9nsd")
output_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/molecules")
training_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/training")
test_dir = Path("/home/rmonge/UPC/TFM/gnn-qm9/data/test")

empty_dirs([output_dir, training_dir, test_dir])
process_files(data_dir=data_dir, limit=limit, output_dir=output_dir)

In [10]:
seed = 42
np.random.seed(seed)

files = np.array(os.listdir(output_dir))
np.random.shuffle(files)
test_files = files[:test_molecules]
training_files = files[test_molecules:]

# Empty training/test dir
for file in os.listdir(training_dir):
    os.remove(os.path.join(training_dir, file))
for file in os.listdir(test_dir):
    os.remove(os.path.join(test_dir, file))

for file in training_files:
    sh.copyfile(os.path.join(output_dir, file), os.path.join(training_dir, file))
for file in test_files:
    sh.copyfile(os.path.join(output_dir, file), os.path.join(test_dir, file))